#AutoEncoders

##Downloading the dataset

##Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

## Importing the dataset


In [2]:
# We won't be using this dataset.
movies = pd.read_csv('movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

## Preparing the training set and the test set


In [3]:
training_set = pd.read_csv('u1.base', delimiter = '\t') 
training_set = np.array(training_set, dtype = 'int') # Transforming the dataframe to np array
test_set = pd.read_csv('u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int') # Transforming the dataframe to np array

## Getting the number of users and movies


In [4]:
nb_users = int(max(max(training_set[:, 0]), max(test_set[:, 0]))) # The number of users that we have in the test set and the training set which is the maximum user ID from the training set
# and the test set 

nb_movies = int(max(max(training_set[:, 1]), max(test_set[:, 1]))) # The number of users that we have in the test set and the training set which is the maximum user ID 
# from the training set and the test set 

## Converting the data into an array with users in lines and movies in columns


In [5]:
def convert(data):
  new_data = []

  for id_users in range(1, nb_users+1):
    id_movies = data[:, 1] [data[:, 0] == id_users] # This a syntax of a conditional statment in pyhton
    
    id_ratings = data[:, 2] [data[:, 0] == id_users] # This a syntax of a conditional statment in pyhton

    ratings = np.zeros(nb_movies) # Here we are initializing a new list of zeros with the size of all the movies that we have so we can replace the zeros
    # with the rating if it is there

    ratings[id_movies - 1] = id_ratings # Here we are trying to build the ratings array by finding the matches for it in the id_ratings so if there is a
    # raing for the movie id mentioned then the ratings will be transfered or equaled otherwise it will still be 0

    new_data.append(list(ratings)) # appending the ratings of all the movies in the users list that will result in a 2d list
  
  return new_data

training_set = convert(training_set)
test_set = convert(test_set)

## Converting the data into Torch tensors


In [6]:
#Remember the tesnor is a multi-dimensional matrix with one type in it 

training_set = torch.FloatTensor(training_set) # We are converting the 2d list of training set into a 2d Floattensor so we can build the architecture of the boltzmann network
# so we can do the training and the building much effecient with tensors

test_set = torch.FloatTensor(test_set) # We are converting the 2d list of test set into a 2d Floattensor so we can build the architecture of the boltzmann network
# so we can do the training and the building much effecient with tensors

In [7]:
print(training_set)

tensor([[0., 3., 4.,  ..., 0., 0., 0.],
        [4., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [5., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 5., 0.,  ..., 0., 0., 0.]])


In [8]:
print(test_set)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


## Creating the architecture of the Neural Network


In [9]:
class SAE(nn.Module):
  def __init__(self):
    super(SAE, self).__init__() # Here we are making sure that we get all the methods and attributes of the class nn.Module
    self.full_connection_1 = nn.Linear(nb_movies, 50) # Here we are making the the first full connection leyer which is the input 
    # layer with dim (nb_movies) for the inputs and outputs to 20 hidden neurons, which are the features extracted from the
    # input layer which decreases the amount of features used depending on features the movies is based on (genre, actors, awards and etc)
    
    self.full_connection_2 = nn.Linear(50, 10) # Here we are making the the second full connection leyer which is the first 
    # hidden layer with dim (20) for the inputs and outputs to 10 hidden neurons, which are the features extracted from the
    # previous layer which decreases the amount of features used depending on features the movies is based on (genre, actors, awards and etc)

    self.full_connection_3 = nn.Linear(10, 50) # Here we are making the the third full connection leyer which is the second 
    # hidden layer with dim (10) for the inputs and outputs to 20 hidden neurons, which are the features extracted from the
    # previous layer and increases the amount of features used becuase now we are entering the decoding phase

    self.full_connection_4 = nn.Linear(50, nb_movies) # Here we are making the the fourth full connection leyer which is the output 
    # layer with dim (20) for the inputs and outputs to (nb_movies) output neurons, which are the features extracted from the
    # previous layer and increases the amount of features used becuase now we are entering the decoding phase and outputing the 
    # xecat amount number of features entered which is the number of movies we have
    
    self.activation = nn.Sigmoid() # Here we are making an attribute that is the essence of training the Auto Encoder

  
  def forward(self, x):
    x = self.activation(self.full_connection_1(x)) # Here we are appltying the sigmoid activation function on the input matrix of features
    # To encode it using the input layer

    x = self.activation(self.full_connection_2(x)) # Here we are appltying the sigmoid activation function on the input matrix of features from
    # the input layer To edncode it using the first hidden layer
    
    x = self.activation(self.full_connection_3(x)) # Here we are appltying the sigmoid activation function on the input matrix of features from
    # the first hidden layer To edncode it using the second hidden layer

    x = self.full_connection_4(x) # Here we are in the output layer where the final decoding layer and procedure happens ,and where the number of 
    # features go back to their original number which is nb_movies

    return x #returning x so we don't modify the original matrix of feature x

sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.6)

## Training the SAE


In [10]:
nb_epoch = 200

for epoch in range(1, nb_epoch+1):
  train_loss = 0
  counter = 0.
  for id_user in range(nb_users):
    inputs = Variable(training_set[id_user]).unsqueeze(0)
    target = inputs.clone()
    if torch.sum(target.data > 0) > 0:
      output = sae(inputs)
      target.require_grad = False
      output[ target==0 ] = 0
      loss = criterion(output, target)
      mean_corrector = nb_movies/float(torch.sum(target.data > 0)+ 1e-10)
      loss.backward() # Decides the drirection of the updated weights
      train_loss += np.sqrt(loss.data * mean_corrector)
      counter += 1.
      optimizer.step() # Decides the intensity of which the weights gets updated
  
  print('epoch: '+str(epoch)+" loss: "+str(train_loss/counter))

epoch: 1 loss: tensor(1.5851)
epoch: 2 loss: tensor(1.0831)
epoch: 3 loss: tensor(1.0652)
epoch: 4 loss: tensor(1.0582)
epoch: 5 loss: tensor(1.0545)
epoch: 6 loss: tensor(1.0527)
epoch: 7 loss: tensor(1.0499)
epoch: 8 loss: tensor(1.0488)
epoch: 9 loss: tensor(1.0453)
epoch: 10 loss: tensor(1.0431)
epoch: 11 loss: tensor(1.0433)
epoch: 12 loss: tensor(1.0378)
epoch: 13 loss: tensor(1.0395)
epoch: 14 loss: tensor(1.0333)
epoch: 15 loss: tensor(1.0356)
epoch: 16 loss: tensor(1.0299)
epoch: 17 loss: tensor(1.0321)
epoch: 18 loss: tensor(1.0255)
epoch: 19 loss: tensor(1.0278)
epoch: 20 loss: tensor(1.0228)
epoch: 21 loss: tensor(1.0214)
epoch: 22 loss: tensor(1.0181)
epoch: 23 loss: tensor(1.0164)
epoch: 24 loss: tensor(1.0158)
epoch: 25 loss: tensor(1.0142)
epoch: 26 loss: tensor(1.0123)
epoch: 27 loss: tensor(1.0096)
epoch: 28 loss: tensor(1.0093)
epoch: 29 loss: tensor(1.0081)
epoch: 30 loss: tensor(1.0072)
epoch: 31 loss: tensor(1.0078)
epoch: 32 loss: tensor(1.0037)
epoch: 33 loss: t

## Testing the SAE


In [11]:
test_loss = 0
counter = 0.
for id_user in range(nb_users):
  inputs = Variable(training_set[id_user]).unsqueeze(0)
  target = Variable(test_set[id_user]).unsqueeze(0)
  if torch.sum(target.data > 0) > 0:
    output = sae(inputs)
    target.require_grad = False
    output[ target==0 ] = 0
    loss = criterion(output, target)
    mean_corrector = nb_movies/float(torch.sum(target.data > 0)+ 1e-10)
    test_loss += np.sqrt(loss.data * mean_corrector)
    counter += 1.

print("test loss: "+str(test_loss/counter))

test loss: tensor(0.9432)
